In [1]:
#!pypy3 -m pip install numpy
#!pypy3 -m pip install tqdm
#!pypy3 -m pip install scalene

In [2]:
import re
import random
import numpy as np
#from tqdm.notebook import tqdm
from tqdm import tqdm
import scalene
import math
import numba as nb

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:

# Define the possible characters in a regex pattern
atoms = '.^$*+?{}[]|()'
metas = 'abcdefghijklm'
atoms = '.^$*+?[]|()'
metas = 'abcdefijklm'

regex_chars = {'1': '\\1', '2': '\\2', '3': '\\3', '!': '{4}', '@': '{2}', '#': '{3}',
               #'4': '\\4',
               '\\':'\\\\', 'n': '(?=', 'o': '(?!', 'p': '(?<!', 'q': '(?<='}
for m, k in zip(metas, atoms):
    regex_chars[k] = k
    regex_chars[m] = '\\'+k

regex_chars_simple = list(regex_chars.keys())

U = '.^$*+?{}[]|()1234'

# Function to generate a random regex pattern
def generate_random_pattern(size=16):
    return ''.join(random.choices(regex_chars_simple, k=size))
# probably works

def fraction_of_match(patt, string):
    match = patt.search(string)
    str_len = len(string)
    return ((match.end() - match.start()) / str_len if match else 0) if str_len else int(bool(match))

# Ensure math.isclose is used for all floating-point comparisons
assert math.isclose(fraction_of_match(re.compile('a'), 'abc'), 1/3), "Match at the start should calculate fraction correctly."
assert math.isclose(fraction_of_match(re.compile('c'), 'abc'), 1/3), "Match at the end should calculate fraction correctly."
assert math.isclose(fraction_of_match(re.compile('d'), 'abc'), 0), "No match should return 0."
assert math.isclose(fraction_of_match(re.compile('a'), ''), 0), "Empty string should return 0 regardless of pattern."
assert math.isclose(fraction_of_match(re.compile(''), 'abc'), 0), "Empty pattern with non-empty string should return 0."
assert math.isclose(fraction_of_match(re.compile('abc'), 'abc'), 1), "Entire string match should return 1."
assert math.isclose(fraction_of_match(re.compile(''), ''), 1), "Both empty should return 0."
assert math.isclose(fraction_of_match(re.compile('b'), 'abc'), 1/3), "Pattern matches part of the string should calculate fraction correctly."
# Additional case: Multiple occurrences only considering the first match
assert math.isclose(fraction_of_match(re.compile('a+|c'), 'aabc'), 1/2), "First match considered in multiple occurrences should calculate fraction correctly."

# Note: The initial implementation might need adjustment if the intention was to handle empty patterns or strings differently.

def transform_patt(pattern):
    return ''.join(regex_chars[k] for k in pattern)
# probably works

# Fitness function to evaluate how close a pattern is to behaving like a quine
def fitness(pattern):
    if len(pattern) == 0 or len(pattern) > 48:
        return 0
    badness = (
        + abs(pattern.count('[')-pattern.count(']'))
        + abs(pattern.count('(')+pattern.count('n')+pattern.count('o')+pattern.count('p')+pattern.count('q')-pattern.count(')'))
        + pattern.count('**') + pattern.count('++') + pattern.count('||') + pattern.count('^^') + pattern.count('??') + pattern.count('\\\\') + pattern.count('..')
    )
    pattern_t = transform_patt(pattern)
    try:
        patt = re.compile(pattern_t)
        # A simple fitness metric: does the pattern match itself and not match a reversed version or random string
        self_match = fraction_of_match(patt, pattern_t)
        empty_match = fraction_of_match(patt, '')
        cover_match = sum(fraction_of_match(patt, '('*i+pattern_t+')'*i) for i in range(10))
        drop_one_match = sum(fraction_of_match(patt, pattern_t[:i] + pattern_t[i+1:]) for i in range(len(pattern_t))) / len(pattern_t)
        replace_one_match = sum(fraction_of_match(patt, pattern_t[:i] + u + pattern_t[i+1:]) for i in range(len(pattern_t)) for u in U) / len(pattern_t)
        return self_match*100 - empty_match - cover_match*20 - drop_one_match*20 - replace_one_match*60  - badness
    except (re.error, OverflowError):
        # Penalize invalid regex patterns
        return -100

def mutate(pattern, base_rate=0.06):
    actions = ['insert', 'delete', 'replace', 'swap']*15
    actions = ['insert']*10 + ['delete']*10 + ['replace']*2 + ['swap']*15
    mutation_rate = base_rate + (0.1 * random.random())  # Slightly vary the mutation rate

    # Apply multiple mutations based on mutation rate
    for _ in range(len(pattern)):
        if random.random() < mutation_rate:
            action = random.choice(actions)

            if action == 'insert':
                position = random.randrange(len(pattern) + 1)
                char = random.choice(regex_chars_simple)
                pattern = pattern[:position] + char + pattern[position:]

            elif action == 'delete' and len(pattern) > 1:
                position = random.randrange(len(pattern))
                pattern = pattern[:position] + pattern[position+1:]

            elif action == 'replace':
                position = random.randrange(len(pattern))
                char = random.choice(regex_chars_simple)
                pattern = pattern[:position] + char + pattern[position+1:]

            elif action == 'swap' and len(pattern) > 2:
                pos1, pos2 = random.sample(range(len(pattern)), 2)
                pattern_list = list(pattern)
                pattern_list[pos1], pattern_list[pos2] = pattern_list[pos2], pattern_list[pos1]
                pattern = ''.join(pattern_list)

    return pattern


# Crossover function to combine two patterns
def crossover(parent1, parent2):
    if len(parent1) < 2 or len(parent2) < 2:
        return parent1, parent2
    children = []
    order = list(range(1, len(parent1)-1))
    order2 = list(range(1, len(parent2)-1))
    random.shuffle(order)
    random.shuffle(order2)
    for c1 in order:
        expressions = []
        canLHS = False
        try:
            re.compile(parent1[:c1])
            expressions.add(parent1[:c1])
        except:
            pass
        try:
            re.compile(parent1[c1:])
            expressions.add(parent1[c1:])
        except:
            pass
        if not expressions:
            continue
        for c2 in order2:
            try:
                re.compile(parent2[c2:])
                children.append(''.join(random.shuffle([parent2[c2:], random.choice(expressions)])))
                if len(children) == 2:
                    return children
            except:
                pass
            try:
                re.compile(parent2[:c2])
                children.append(''.join(random.shuffle([parent2[:c2], random.choice(expressions)])))
                if len(children) == 2:
                    return children
            except:
                pass
    if len(children) == 1:
        return children[0] + random.choice([parent1, parent2])
    return parent1, parent2

def evolve_population(population_size=2000, generations=500, elitism_fraction=0.25):
    # Initialize population
    population = [generate_random_pattern() for _ in range(population_size)]
    
    for generation in range(generations):
        # Evaluate fitness
        fitness_scores = [fitness(p) for p in population]
        
        # Sort population by fitness in descending order
        sorted_indices = np.argsort(fitness_scores)[::-1]
        sorted_population = [population[i] for i in sorted_indices]
        
        # Select the top 25% performers
        elite_count = int(population_size * elitism_fraction)
        elites = sorted_population[:elite_count]
        
        # Print best pattern and fitness every few generations
        if generation % 50 == 0 or generation == generations - 1:
            best_pattern = elites[0]
            print(f"Generation {generation}: Best Pattern: '{transform_patt(best_pattern)}' with Fitness: {fitness_scores[sorted_indices[0]]}")

        # Crossover and mutation to generate the rest of the next generation
        next_generation = elites.copy()  # Start with the elites
        while len(next_generation) < population_size:
            parent1, parent2 = random.sample(sorted_population[:elite_count + (population_size//4)], 2)  # Sample from the top 50%
            child1, child2 = crossover(parent1, parent2)
            next_generation.append(mutate(child1))
            if len(next_generation) < population_size:
                next_generation.append(mutate(child2))

        population = next_generation

    return elites[0]  # Return the best pattern found

# Run the evolutionary algorithm
best_regex = evolve_population()
print(f"Best evolved regex pattern: {transform_patt(best_regex)}")


Generation 0: Best Pattern: '[\)\(.\\\1\[]*$\*\)$\\\)\[' with Fitness: 0.0
Generation 50: Best Pattern: '\[\[\+*\^*?\({3}' with Fitness: 0.0
Generation 100: Best Pattern: '\|^' with Fitness: 0.0


KeyboardInterrupt: 